**T&D - Formatos de tabela**
---------------
O objetivo deste notebook é importar dados em formato JSON e utiliza-los para formar tabelas e comparar o tempo de salvamento e o tamanho daquele determinado formato.

Serão comparados os tipos:
- Delta
- Hudi: Merge_on_read & ORC base

# Imports & Definitions

In [1]:
import pyspark
import pyspark.sql.functions as F
import os
import pandas as pd

In [2]:

cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
os.chdir(parent_dir)

In [3]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder 
    .master("local[*]")
    .appName("default")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/17 21:10:39 WARN Utils: Your hostname, quero-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.15.151 instead (on interface wlp2s0)
25/06/17 21:10:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/eric-yoshida/.ivy2/cache
The jars for the packages stored in: /home/eric-yoshida/.ivy2/jars


:: loading settings :: url = jar:file:/home/eric-yoshida/pyspark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2fb5549f-28d7-4c3b-8e54-a57265c0d648;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 181ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.2 from central in [default]
	io.delta#delta-storage;3.3.2 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: r

In [4]:
data_path = "./data/raw/"

kindle_df = spark.read.json(data_path + "Kindle_Store_5.json.gz")
movies_df = spark.read.json(data_path + "Movies_and_TV_5.json.gz")

In [5]:
for kindle_field, movie_field in zip(kindle_df.schema.fields, movies_df.schema.fields):
    if kindle_field == movie_field:
        print(f"Fields match: {kindle_field}")
    else:
        print(f"Fields do not match: {kindle_field} != {movie_field}")

Fields match: StructField('asin', StringType(), True)
Fields match: StructField('image', ArrayType(StringType(), True), True)
Fields match: StructField('overall', DoubleType(), True)
Fields match: StructField('reviewText', StringType(), True)
Fields match: StructField('reviewTime', StringType(), True)
Fields match: StructField('reviewerID', StringType(), True)
Fields match: StructField('reviewerName', StringType(), True)
Fields do not match: StructField('style', StructType([StructField('Format:', StringType(), True)]), True) != StructField('style', StructType([StructField('Format:', StringType(), True), StructField('Shape:', StringType(), True), StructField('Size:', StringType(), True)]), True)
Fields match: StructField('summary', StringType(), True)
Fields match: StructField('unixReviewTime', LongType(), True)
Fields match: StructField('verified', BooleanType(), True)
Fields match: StructField('vote', StringType(), True)


In [6]:
kindle_df = kindle_df.withColumn("style", F.split(F.col("style.`Format:`"), ",").getItem(0))
movies_df = movies_df.withColumn("style", F.split(F.col("style.`Format:`"), ",").getItem(0))

In [7]:
movies_df.limit(10).toPandas()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,0005089549,None,5.0,So sorry I didn't purchase this years ago when...,"11 9, 2012",A2M1CU2IRZG0K9,Terri,VHS Tape,Amazing!,1352419200,True,None
1,0005089549,None,5.0,Believe me when I tell you that you will recei...,"12 30, 2011",AFTUJYISOFHY6,Melissa D. Abercrombie,VHS Tape,Great Gospel VHS of the Cathedrals!,1325203200,True,None
2,000503860X,None,5.0,"I have seen X live many times, both in the ear...","04 21, 2005",A3JVF9Y53BEOGC,Anthony Thompson,DVD,A great document of a great band,1114041600,True,11
3,000503860X,None,5.0,"I was so excited for this! Finally, a live co...","04 6, 2005",A12VPEOEZS1KTC,JadeRain,DVD,YES!! X LIVE!!,1112745600,True,5
4,000503860X,None,5.0,X is one of the best punk bands ever. I don't ...,"12 3, 2010",ATLZNVLYKP9AZ,T. Fisher,DVD,X have still got it,1291334400,True,5
5,000503860X,None,5.0,I've loved X since I first saw them here in Sa...,"10 17, 2007",A3TNYNA2360NPA,Stanley C. Sargent,DVD,A Revival to match the early shows by a great ...,1192579200,True,None
6,000503860X,None,5.0,I must admit I was hesitant to purchase this D...,"11 8, 2005",A2LUL6PRTXE7SE,PSM/Bokor,DVD,Old punks still rule...,1131408000,True,17
7,0005419263,None,5.0,The little ones love this,"04 12, 2016",A2CFV9UPFTTM10,SuzieQ,Audio CD,Love it,1460419200,True,None
8,0005419263,None,3.0,Good songs. The DVD is a little boring.,"07 28, 2014",A3139J3877Y61F,SingingButterfly,Audio CD,Three Stars,1406505600,True,None
9,0005419263,None,5.0,The DVD came in great condition and provided l...,"12 21, 2010",A2PANT8U0OJNT4,Cutegirlmomma,DVD,Blast from the past,1292889600,True,None


In [8]:
kindle_df.limit(10).toPandas()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,B000FA5KK0,None,4.0,"pretty good story, a little exaggerated, but I...","07 3, 2014",A2LSKD2H9U8N0J,sandra sue marsolek,Kindle Edition,pretty good story,1404345600,True,None
1,B000FA5KK0,None,5.0,"If you've read other max brand westerns, you k...","05 26, 2014",A2QP13XTJND1QS,Tpl,Kindle Edition,A very good book,1401062400,True,None
2,B000FA5KK0,None,5.0,"Love Max, always a fun twist","09 16, 2016",A8WQ7MAG3HFOZ,Alverne F. Anderson,Kindle Edition,Five Stars,1473984000,True,None
3,B000FA5KK0,None,5.0,"As usual for him, a good book","03 3, 2016",A1E0MODSRYP7O,Jeff,Kindle Edition,a good,1456963200,True,None
4,B000FA5KK0,None,5.0,MB is one of the original western writers and ...,"09 10, 2015",AYUTCGVSM1H7T,DEHS - EddyRapcon,Kindle Edition,A Western,1441843200,True,2
5,B000FA5KK0,None,5.0,great book,"06 14, 2015",A9DXGM6YGV14D,Linda DeWachter,Kindle Edition,Five Stars,1434240000,True,None
6,B000FA5KK0,None,3.0,"A good, solid Western - yes, a little contrive...","06 2, 2015",A3MF8G33UKQLGL,mzbeastle,Kindle Edition,"Fast read, entertaining, like a 1930s Western ...",1433203200,True,3
7,B000FA5KK0,None,4.0,ALMOST BEEN TOO LONG SINCE I READ IT. GOOD REA...,"05 3, 2015",A12AIK6DSUF1EW,R. DAVIS,Kindle Edition,ANOTHER GOOD WESTERN,1430611200,True,None
8,B000FA5KK0,None,5.0,Enjoyed this book and will read more from this...,"02 2, 2015",A2RW1CXT3XSLXW,Barbara Rigsby,Kindle Edition,Good,1422835200,True,None
9,B000FA5KK0,None,4.0,A very good read for you Western fans. Great ...,"12 19, 2014",A2ID7H6SRK5XS0,Richard Johnson,Kindle Edition,Get it,1418947200,True,None


In [9]:
united_df = (
    kindle_df
    .union(movies_df)
)

In [ ]:
united_df.write.json

# Saving as Delta

In [19]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder 
    .master("local[*]")
    .appName("delta")
    .config("spark.jars.packages",
            "io.delta:delta-core_2.12:3.3.2")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [20]:
delta_path = "./data/delta/united_data"

united_df.write.format("delta").mode("overwrite").option("overwriteSchema", "True").save(delta_path)

AnalysisException: [DELTA_CONFIGURE_SPARK_SESSION_WITH_EXTENSION_AND_CATALOG] This Delta operation requires the SparkSession to be configured with the
DeltaSparkSessionExtension and the DeltaCatalog. Please set the necessary
configurations when creating the SparkSession as shown below.

  SparkSession.builder()
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    ...
    .getOrCreate()

If you are using spark-shell/pyspark/spark-submit, you can add the required configurations to the command as show below:
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog


In [ ]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder 
    .master("local[*]")
    .appName("hudi")
    .config("spark.jars", "/home/eric-yoshida/.ivy2/jars/hudi-spark3.5-bundle_2.12-1.0.2.jar")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/17 20:52:49 WARN Utils: Your hostname, quero-Inspiron-15-3520 resolves to a loopback address: 127.0.1.1; using 192.168.15.151 instead (on interface wlp2s0)
25/06/17 20:52:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/eric-yoshida/pyspark-env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/eric-yoshida/.ivy2/cache
The jars for the packages stored in: /home/eric-yoshida/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47d3ee2d-4240-47de-b64d-6c35f992eab4;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 200ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.2 from central in [default]
	io.delta#delta-storage;3.3.2 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     | 

In [26]:
united_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [27]:
hudi_path = "./data/hudi/united_data"

(
    united_df.write
        .format("hudi")
        .option("hoodie.datasource.write.table.type", "MERGE_ON_READ")
        .option("hoodie.base.file.format", "ORC")
        .option("hoodie.datasource.write.recordkey.field", "reviewerID")
        .option("hoodie.datasource.write.precombine.field", "unixReviewTime")
        .mode("overwrite")
        .option("overwriteSchema", "True")
        .save(hudi_path)
 )

Py4JJavaError: An error occurred while calling o249.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: hudi. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:873)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:260)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:243)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.inter

In [ ]:
spark.stop()